In [ ]:
"""
CAPTCHA Generator Module
Handles: random text + image generation
"""

import io
import random
import string
import secrets
from PIL import Image, ImageDraw, ImageFont, ImageFilter

# Configuration
CAPTCHA_LENGTH = 6
IMAGE_WIDTH = 220
IMAGE_HEIGHT = 80
FONT_SIZE = 40
FONT_PATHS = ["arial.ttf", "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf"]


def random_text(length=CAPTCHA_LENGTH):
    """Generate a secure random CAPTCHA string using uppercase letters and digits."""
    alphabet = string.ascii_uppercase + string.digits
    confusing = {'0', 'O', '1', 'I', '5', 'S'}
    out = []
    while len(out) < length:
        c = secrets.choice(alphabet)
        if c in confusing:
            continue
        out.append(c)
    return ''.join(out)


def load_font(size=FONT_SIZE):
    """Try to load a system font; fallback to default."""
    for p in FONT_PATHS:
        try:
            return ImageFont.truetype(p, size)
        except Exception:
            continue
    return ImageFont.load_default()


def create_captcha_image(text, width=IMAGE_WIDTH, height=IMAGE_HEIGHT):
    """Create a CAPTCHA image from text with random noise and distortion."""
    bg_color = (255, 255, 255)
    image = Image.new('RGB', (width, height), bg_color)
    draw = ImageDraw.Draw(image)

    # Background noise
    for _ in range(random.randint(150, 250)):
        x1, y1 = random.randint(0, width), random.randint(0, height)
        x2, y2 = x1 + random.randint(1, 6), y1 + random.randint(1, 6)
        color = tuple(random.randint(150, 255) for _ in range(3))
        draw.rectangle([x1, y1, x2, y2], fill=color)

    font = load_font()
    char_width = width // (len(text) + 1)
    baseline = (height - FONT_SIZE) // 2

    # Draw characters
    for i, ch in enumerate(text):
        x = char_width * (i + 1) - FONT_SIZE // 2 + random.randint(-5, 5)
        y = baseline + random.randint(-10, 10)
        fill = tuple(random.randint(10, 120) for _ in range(3))
        char_img = Image.new('RGBA', (FONT_SIZE * 2, FONT_SIZE * 2), (0, 0, 0, 0))
        char_draw = ImageDraw.Draw(char_img)
        char_draw.text((10, 10), ch, font=font, fill=fill)
        rotated = char_img.rotate(random.uniform(-30, 30), resample=Image.BICUBIC, expand=1)
        image.paste(rotated, (x, y), rotated)

    # Lines + Blur
    for _ in range(random.randint(4, 8)):
        draw.line(
            [random.randint(0, width), random.randint(0, height),
             random.randint(0, width), random.randint(0, height)],
            fill=tuple(random.randint(0, 150) for _ in range(3)),
            width=random.randint(1, 3)
        )

    image = image.filter(ImageFilter.SMOOTH).filter(ImageFilter.GaussianBlur(radius=0.5))
    return image


def generate_captcha():
    """Return both the text and the image (as bytes)."""
    text = random_text()
    img = create_captcha_image(text)
    buf = io.BytesIO()
    img.save(buf, format='PNG')
    buf.seek(0)
    return text, buf
